<a href="https://colab.research.google.com/github/aarsanjani/meansquares/blob/master/MobilityFactor_ProximityMetricAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data source  

* https://github.com/ActiveConclusion/COVID19_mobility

## Google Mobility Data 

In [ ]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=2e278fd07d6379402d1983d1f175d18e0ffea0d663d0e7ab729e7f011e7a0b91
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
import wget
from pandas import Series, datetime
from pandas.plotting import scatter_matrix, autocorrelation_plot
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import roc_curve, auc
import random
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from xgboost import XGBClassifier
from sklearn.mixture import GaussianMixture

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
google_data_url = 'https://raw.githubusercontent.com/ActiveConclusion/COVID19_mobility/master/google_reports/mobility_report_US.csv'

google_filename = wget.download(google_data_url)

In [ ]:
google_mobility_data = pd.read_csv(google_filename,low_memory=False)
google_mobility_data.head(2)

,state,county,date,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential
0,Total,Total,2020-02-15,6.0,2.0,15.0,3.0,2.0,-1.0
1,Total,Total,2020-02-16,7.0,1.0,16.0,2.0,0.0,-1.0


Note: This dataset has only US data, hence I am filtering state 'California' 

In [ ]:
CA_GoogleMobility_data = google_mobility_data[google_mobility_data['state'] == 'California']
print(CA_GoogleMobility_data.shape)
CA_GoogleMobility_data.tail(2)

(9981, 9)


,state,county,date,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential
37857,California,Yuba County,2020-08-08,-11.0,17.0,NaN,NaN,-15.0,2.0
37858,California,Yuba County,2020-08-09,-1.0,19.0,NaN,NaN,-18.0,1.0


### Data Information

Google captures People's purpose of visit

* Retail and Recreation
* Grocery and Pharmacy
* Parks
* Transit stations
* Workplaces
* Residential


**Google note:** These Community Mobility Reports aim to provide insights into what has changed in response to policies aimed at combating COVID-19. The reports chart movement trends over time by geography, across different categories of places such as retail and recreation, groceries and pharmacies, parks, transit stations, workplaces, and residential.

## Apple Mobility data 

In [ ]:
apple_report_url = 'https://raw.githubusercontent.com/ActiveConclusion/COVID19_mobility/master/apple_reports/apple_mobility_report_US.csv'

apple_filename = wget.download(apple_report_url)

In [ ]:
apple_mobility_data = pd.read_csv(apple_filename,low_memory=False)
apple_mobility_data.tail(2)

,state,county_and_city,geo_type,date,driving,transit,walking
473758,Wyoming,Uinta County,county,2020-08-10,259.61,NaN,NaN
473759,Wyoming,Uinta County,county,2020-08-11,258.68,NaN,NaN


In [ ]:
CA_AppleMobility_data = apple_mobility_data[apple_mobility_data['state'] == 'California']
print(CA_AppleMobility_data.shape)
CA_AppleMobility_data.tail(2)


(15750, 7)


,state,county_and_city,geo_type,date,driving,transit,walking
42838,California,Yuba County,county,2020-08-10,65.03,NaN,NaN
42839,California,Yuba County,county,2020-08-11,60.60,NaN,NaN


### Data information

Apple mobility data has the following columns
* driving
* transit
* walking

**Apple note**: The CSV file shows a relative volume of directions requests per country/region or city compared to a baseline volume on January 13th, 2020.

Day defined as midnight-to-midnight, Pacific time. Cities represent usage in greater metropolitan areas and are stably defined during this period. In many countries/regions and cities, relative volume has increased since January 13th, consistent with normal, seasonal usage of Apple Maps. Day of week effects are important to normalize as you use this data. 

## Amalgamation of Google and Apple data

In [ ]:
merged_data = pd.concat()